**Goal**
Understand the size, coverage, and basic patterns in the NYPD Complaint Data so we can  build crime-based neighborhood features around restaurants.


**Plan**
1. Load the NYPD complaint CSV from `data/raw`.
2. Keep only the columns we need (ID, date/time, borough, offense, lat/long).
3. Convert dates and numeric fields to the right types and handle missing values.
4. Filter out invalid records
5. Summarize the cleaned data: size, date range, borough counts, top offense types.
6. Save a cleaned version
7. Write a short summary of data

In [2]:
import pandas as pd

nypd_path = "../data/raw/NYPD_Complaint_Data_Historic.csv"
nypd = pd.read_csv(nypd_path)

/var/folders/_l/_85pbswx3gb9p0649jw7x4rm0000gn/T/ipykernel_55824/1405498160.py:4: DtypeWarning: Columns (0,20) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd = pd.read_csv(nypd_path)


In [5]:
nypd.head(20)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,39468181,02/20/2008,07:00:00,02/23/2008,08:00:00,88.0,02/23/2008,107,BURGLARY,221.0,...,(null),NaN,40.692464,-73.972708,"(40.692464, -73.972708)",PATROL BORO BKLYN NORTH,(null),25-44,WHITE,F
1,50539499,08/21/2008,22:00:00,08/21/2008,23:00:00,19.0,08/22/2008,109,GRAND LARCENY,438.0,...,(null),NaN,40.771341,-73.953418,"(40.771341, -73.953418)",PATROL BORO MAN NORTH,(null),45-64,WHITE HISPANIC,F
2,45223390,04/03/2008,03:35:00,04/03/2008,03:50:00,77.0,04/03/2008,106,FELONY ASSAULT,109.0,...,(null),NaN,40.671245,-73.926713,"(40.671245, -73.926713)",PATROL BORO BKLYN NORTH,(null),25-44,BLACK,F
3,50594658,08/19/2008,09:00:00,NaN,(null),32.0,08/27/2008,341,PETIT LARCENY,349.0,...,(null),NaN,40.813412,-73.943226,"(40.813412, -73.943226)",PATROL BORO MAN NORTH,(null),(null),UNKNOWN,M
4,44451016,03/10/2008,22:00:00,03/10/2008,22:10:00,67.0,03/11/2008,105,ROBBERY,397.0,...,M,NaN,40.650142,-73.944674,"(40.650142, -73.944674)",PATROL BORO BKLYN SOUTH,(null),25-44,BLACK,M
5,49918184,08/08/2008,01:30:00,08/08/2008,01:34:00,34.0,08/08/2008,355,OFFENSES AGAINST THE PERSON,115.0,...,(null),NaN,40.855233,-73.929406,"(40.855233, -73.929406)",PATROL BORO MAN NORTH,(null),(null),UNKNOWN,E
6,55066472,12/21/2008,06:07:00,NaN,(null),78.0,12/21/2008,344,ASSAULT 3 & RELATED OFFENSES,101.0,...,M,NaN,40.669126,-73.973071,"(40.669126, -73.973071)",PATROL BORO BKLYN SOUTH,(null),25-44,WHITE,M
7,55244936,12/27/2008,02:04:00,NaN,(null),23.0,12/27/2008,347,INTOXICATED & IMPAIRED DRIVING,905.0,...,(null),NaN,40.791570,-73.944677,"(40.79156983, -73.94467676)",PATROL BORO MAN NORTH,(null),(null),UNKNOWN,E
8,47391386,06/19/2008,15:30:00,06/19/2008,16:30:00,61.0,06/20/2008,352,CRIMINAL TRESPASS,205.0,...,(null),NaN,40.605188,-73.952509,"(40.605188, -73.952509)",PATROL BORO BKLYN SOUTH,(null),18-24,WHITE HISPANIC,F
9,45814265,04/19/2008,19:45:00,04/19/2008,19:50:00,83.0,04/19/2008,578,HARRASSMENT 2,637.0,...,M,NaN,40.689954,-73.916924,"(40.689954, -73.916924)",PATROL BORO BKLYN NORTH,(null),25-44,WHITE HISPANIC,F
